# Выбор локации для скважины

Добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 

**Цель исследования**: построить модель для определения региона, где добыча принесёт наибольшую прибыль и проанализировать возможную прибыль и риски техникой Bootstrap.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
INCOME_PER_UNIT = 450000   #доход за единицу
POINTS = 500               #кол-во точек для разработки
BEST_POINTS = 200          #кол-во лучших точек для разработки

#BUDGET = 10e10             #бюджет

BUDGET = 1e10 #код ревьюера

NUM = 1000                 #кол-во выборок

In [3]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

### Изучение данных первого региона 

In [4]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
data_1.duplicated().sum()

0

In [7]:
data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

В 1 регионе все данные в порядке, столбцы соостветвуют своим правильным типам данных, отсутсвуют дубликаты и пустые значения.

### Изучение данных второго региона 

In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [10]:
data_2.duplicated().sum()

0

In [11]:
data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Во 2 регионе также все данные в порядке, столбцы соостветвуют своим правильным типам данных, отсутсвуют дубликаты и пустые значения.

### Изучение данных третьего региона 

In [12]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [14]:
data_3.duplicated().sum()

0

In [15]:
data_3.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

В 3 регионе также все данные в порядке, столбцы соостветвуют своим правильным типам данных, отсутсвуют дубликаты и пустые значения.

**Вывод:** дополнительной предобработки данные не требуют. 

## Обучение и проверка модели

### Деление данных на выборки

#### Регион 1

In [16]:
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
print('Размер тренировочной выборки:', data_1_train.shape)
print('Размер валидационной выборки:', data_1_valid.shape)

Размер тренировочной выборки: (75000, 5)
Размер валидационной выборки: (25000, 5)


In [17]:
features_train_1 = data_1_train.drop(['id', 'product'], axis=1)
target_train_1 = data_1_train['product']
features_valid_1 = data_1_valid.drop(['id', 'product'], axis=1)
target_valid_1 = data_1_valid['product']

#### Регион 2

In [18]:
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)
print('Размер тренировочной выборки:', data_2_train.shape)
print('Размер валидационной выборки:', data_2_valid.shape)

Размер тренировочной выборки: (75000, 5)
Размер валидационной выборки: (25000, 5)


In [19]:
features_train_2 = data_2_train.drop(['id', 'product'], axis=1)
target_train_2 = data_2_train['product']
features_valid_2 = data_2_valid.drop(['id', 'product'], axis=1)
target_valid_2 = data_2_valid['product']

#### Регион 3

In [20]:
data_3_train, data_3_valid = train_test_split(data_3, test_size=0.25, random_state=12345)
print('Размер тренировочной выборки:', data_3_train.shape)
print('Размер валидационной выборки:', data_3_valid.shape)

Размер тренировочной выборки: (75000, 5)
Размер валидационной выборки: (25000, 5)


In [21]:
features_train_3 = data_3_train.drop(['id', 'product'], axis=1)
target_train_3 = data_3_train['product']
features_valid_3 = data_3_valid.drop(['id', 'product'], axis=1)
target_valid_3 = data_3_valid['product']

### Обучение модели и расчет метрик

In [22]:
def learning_model(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predicted)**0.5
    print('Среднеквадратичная ошибка RMSE:', rmse)
    print('Cредний запас предсказанного сырья:', predicted.mean())
    return model, predicted, rmse

#### Регион 1

In [23]:
model_1, predicted_1, rmse_1 = learning_model(features_train_1, features_valid_1, target_train_1, target_valid_1)

Среднеквадратичная ошибка RMSE: 37.5794217150813
Cредний запас предсказанного сырья: 92.59256778438035


#### Регион 2

In [24]:
model_2, predicted_2, rmse_2 = learning_model(features_train_2, features_valid_2, target_train_2, target_valid_2)

Среднеквадратичная ошибка RMSE: 0.893099286775617
Cредний запас предсказанного сырья: 68.728546895446


#### Регион 3

In [25]:
model_3, predicted_3, rmse_3 = learning_model(features_train_3, features_valid_3, target_train_3, target_valid_3)

Среднеквадратичная ошибка RMSE: 40.02970873393434
Cредний запас предсказанного сырья: 94.96504596800489


**Вывод:** мы обучили модели и предсказали запасы сырья для всех 3 регионов. Заметны отличия во 2 регионе. Тогда как в 1 и 3 регионе средний запас предсказанного сырья равняется около 93, то во 2 он составляет ~ 69. И среднеквадратичная ошибка во 2 регионе так же сильно отличается, она достаточно небольшая (меньше 1). 

## Подготовка к расчёту прибыли

### Ключевые значения

### Рассчет достаточного объём сырья для безубыточной разработки и сравнение со средним

In [26]:
suff_volume = BUDGET/BEST_POINTS/INCOME_PER_UNIT
f'{suff_volume} тыс. баррелей.'

'111.11111111111111 тыс. баррелей.'

#### Регион 1

In [27]:
print(f'Разница составляет {suff_volume-data_1["product"].mean()} тыс. баррелей.')

Разница составляет 18.6111111111111 тыс. баррелей.


#### Регион 2

In [28]:
print(f'Разница составляет {suff_volume-data_2["product"].mean()} тыс. баррелей.')

Разница составляет 42.2861111111111 тыс. баррелей.


#### Регион 3

In [29]:
print(f'Разница составляет {suff_volume-data_3["product"].mean()} тыс. баррелей.')

Разница составляет 16.11111111111107 тыс. баррелей.


**Вывод:** объём сырья для безубыточной разработки новой скважины, чтобы быть достаточным, должен составлять не менее 111.(1) тыс. баррелей.
* Разница между достаточным объемом сырья и средним запасом для 1 региона составляет 18.6111111111111 тыс. баррелей.
* Разница между достаточным объемом сырья и средним запасом для 2 региона составляет 42.2861111111111 тыс. баррелей.
* Разница между достаточным объемом сырья и средним запасом для 3 региона составляет 16.11111111111107 тыс. баррелей.

## Расчёт прибыли и рисков 

### Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

* Выбераем скважины с максимальными значениями предсказаний. 
* Суммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
* Рассчитываем прибыль для полученного объёма сырья.

In [30]:
def revenue(target, predictions, count):
    probs_sorted = pd.Series(predictions, index=target.index).sort_values(ascending=False)[:count]    
    selected = target[probs_sorted.index].sum() 
    profit_pred = selected * INCOME_PER_UNIT - BUDGET
    return profit_pred

#### Регион 1

In [31]:
profit_pred = revenue(target_valid_1, predicted_1, BEST_POINTS)
print('Прибыль для полученного объёма сырья:', profit_pred, 'тыс. рублей.')

Прибыль для полученного объёма сырья: 3320826043.1398506 тыс. рублей.


#### Регион 2

In [32]:
profit_pred = revenue(target_valid_2, predicted_2, BEST_POINTS)
print('Прибыль для полученного объёма сырья:', profit_pred, 'тыс. рублей.')

Прибыль для полученного объёма сырья: 2415086696.681511 тыс. рублей.


#### Регион 3

In [33]:
profit_pred = revenue(target_valid_3, predicted_3, BEST_POINTS)
print('Прибыль для полученного объёма сырья:', profit_pred, 'тыс. рублей.')

Прибыль для полученного объёма сырья: 2710349963.5998325 тыс. рублей.


### Bootstrap с 1000 выборок

In [34]:
targets = [target_valid_1, target_valid_2, target_valid_3]
predictions = [pd.Series(predicted_1, index=target_valid_1.index), pd.Series(predicted_2, index=target_valid_2.index), pd.Series(predicted_3, index=target_valid_3.index)]
state = np.random.RandomState(12345)

for i in range(3):
    values = []
    for _ in range(NUM):
        target_subsample = targets[i].sample(n=POINTS, replace=True, random_state=state)
        predictions_subsample = predictions[i][target_subsample.index]
        values.append(revenue(target_subsample, predictions_subsample, BEST_POINTS))
    
    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    val_mean = values.mean()
    risk_losses = sum(values<0) / values.count()*100
    
    print(f'Регион {i+1}')
    print('Средняя прибыль:', val_mean)
    print(f'95%-доверительный интервал: от {lower} до {upper}')
    print(f'Риск убытков: {risk_losses}%')
    print()

Регион 1
Средняя прибыль: 600735244.2611653
95%-доверительный интервал: от 12948331.135115242 до 1231163605.7914982
Риск убытков: 2.0%

Регион 2
Средняя прибыль: 663958995.2601906
95%-доверительный интервал: от 206476361.25177094 до 1191197684.7488434
Риск убытков: 0.1%

Регион 3
Средняя прибыль: 597381047.9005232
95%-доверительный интервал: от 1734929.5311760982 до 1246217960.1652355
Риск убытков: 2.5%



Судя по наименьшим рискам подходит регион 1 и 3. Однако средняя прибыль значительно выше во 2 регионе, риски не настолько велики в этом регионе, чтобы ради них жертвовать большей прибыли. Оптимально остановится на регионе 2.

## Вывод

* На начальном этапе исследования были подготовлены данные, которые не требовали предобработки. Затем для каждого региона данные были разбиты на тренировочную и валидационную выборку в соотношении 3:1 соответственно. Для каждого региона обучена модель, для этого использовалась линейная регрессия. 
* Посчитаны осоновные метрики: RMSE, cредний запас предсказанного сырья. Заметны отличия во 2 регионе. Тогда как в 1 и 3 регионе средний запас предсказанного сырья равняется около 93, то во 2 он составляет ~ 69. И среднеквадратичная ошибка во 2 регионе так же сильно отличается, она достаточно небольшая (меньше 1).
* Был сделан вывод, что объём сырья для безубыточной разработки новой скважины, чтобы быть достаточным, должен составлять не менее 111.(1) тыс. баррелей.
* Была посчитна разница между достаточным объемом сырья и средним запасом для каждого региона.
* Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели показала прибыль для полученного объёма сырья для каждого региона.
* С помощью техники Bootstrap с 1000 выборок было найдено распределение прибыли, средняя прибыль, 95%-й доверительный интервал и риск убытков.
* По факторам рисков и средней прибыли было принято решение **выбрать 2 регион** для разработки скважин.